<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_breakdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-10-12 15:41:14--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  31.5MB/s    in 0.2s    

2022-10-12 15:41:14 (31.5 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-10-12 15:41:14--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.9 MB/s 
time: 631 µs (started: 2022-10-12 15:41:25 +00:00)


In [ ]:
! pip install -q deepmatch

     |████████████████████████████████| 147 kB 7.3 MB/s 
     |████████████████████████████████| 2.9 MB 42.1 MB/s 
time: 8.78 s (started: 2022-10-12 15:41:25 +00:00)


In [ ]:
import pandas as pd
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

time: 5.08 s (started: 2022-10-12 15:41:34 +00:00)


In [ ]:
%load_ext autoreload
%autoreload 2

time: 95.1 ms (started: 2022-10-12 15:41:39 +00:00)


# Data

In [ ]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user)#.iloc[:10000] # taking 1w
data = data[data['user_id'].isin([4169,1680,4277])]
print(data.shape)
print(data.user_id.nunique())
print(data.movie_id.nunique())
data.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(5907, 10)
3
3018


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
126387,1680,1193,5,974710779,One Flew Over the Cuckoo's Nest (1975),Drama,M,25,20,95380
126388,1680,661,4,974790436,James and the Giant Peach (1996),Animation,M,25,20,95380


time: 6.1 s (started: 2022-10-12 15:59:02 +00:00)


# Feature Engineering

In [ ]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


time: 50.3 ms (started: 2022-10-12 15:59:11 +00:00)


In [ ]:
i_lbe = LabelEncoder()
u_lbe = LabelEncoder()

time: 56.4 ms (started: 2022-10-12 15:59:12 +00:00)


In [ ]:
data['movie_id']

126387    1193
126388     661
126389     914
126390    2355
126391    1197
          ... 
322342    1364
322343    2444
322344    1770
322345     874
322346    3336
Name: movie_id, Length: 5907, dtype: int64

time: 90 ms (started: 2022-10-12 16:01:57 +00:00)


In [ ]:
item_trans = i_lbe.fit(data['movie_id'])
user_trans = u_lbe.fit(data['user_id'])

time: 72.4 ms (started: 2022-10-12 16:01:06 +00:00)


In [ ]:
item_trans.transform([1193])

array([898])

time: 84 ms (started: 2022-10-12 16:08:35 +00:00)


In [ ]:
item_trans.

{}

time: 61.3 ms (started: 2022-10-12 17:10:38 +00:00)


In [ ]:
item_trans.inverse_transform([898])

array([1193])

time: 96.6 ms (started: 2022-10-12 16:08:49 +00:00)


In [ ]:
data['movie_id'].min()

1

time: 216 ms (started: 2022-10-12 16:48:47 +00:00)


In [ ]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

time: 61.1 ms (started: 2022-10-12 16:03:37 +00:00)


In [ ]:
feature_max_idx

{'movie_id': 3019,
 'user_id': 4,
 'gender': 2,
 'age': 4,
 'occupation': 4,
 'zip': 4,
 'genres': 19}

time: 46.7 ms (started: 2022-10-12 16:03:43 +00:00)


In [ ]:
data.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
126387,1,899,5,974710779,One Flew Over the Cuckoo's Nest (1975),8,1,1,3,2
126388,1,534,4,974790436,James and the Giant Peach (1996),3,1,1,3,2


time: 166 ms (started: 2022-10-12 15:41:52 +00:00)


In [ ]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')
user_profile.shape, item_profile.shape

((3, 4), (3018, 1))

time: 96.5 ms (started: 2022-10-12 15:41:52 +00:00)


In [ ]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)
user_item_list.head()

user_id
1    [899, 534, 679, 1783, 902, 2153, 493, 684, 494...
2    [899, 534, 679, 2599, 902, 990, 2153, 493, 684...
3    [899, 534, 679, 1783, 902, 990, 2153, 493, 684...
Name: movie_id, dtype: object

time: 106 ms (started: 2022-10-12 15:41:52 +00:00)


In [ ]:
train_set, test_set = gen_data_set(data, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 3/3 [00:00<00:00,  9.52it/s]

7 8
time: 441 ms (started: 2022-10-12 15:41:52 +00:00)


In [ ]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

time: 46.4 ms (started: 2022-10-12 15:42:25 +00:00)


In [ ]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((11802,), (11802,), (3,), (3,))

time: 259 ms (started: 2022-10-12 15:42:25 +00:00)


# Create Embeddings

In [ ]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['movie_id'])]
len(train_counter), type(train_counter), len(item_count)

(3018, collections.Counter, 3019)

time: 51.2 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

time: 47.3 ms (started: 2022-10-12 15:42:26 +00:00)


### Create Feature Embeddings



In [ ]:
# item features
item_feature_columns = ['movie_id']
# User features
user_feature_columns = ['user_id',
                        "gender",
                        "age",
                        "occupation",
                        "zip",
                        'hist_movie_id',
                        'hist_genres',
                        'hist_len'
                        ]


time: 48.4 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
feature_max_idx

{'movie_id': 3019,
 'user_id': 4,
 'gender': 2,
 'age': 4,
 'occupation': 4,
 'zip': 4,
 'genres': 19}

time: 45.2 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
user_sparse_feature_columns = user_feature_columns[0:5]
user_sparse_feature_columns

['user_id', 'gender', 'age', 'occupation', 'zip']

time: 50.9 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
user_seq_sparse_feature_columns = user_feature_columns[5:-1]
user_seq_sparse_feature_columns 

['hist_movie_id', 'hist_genres']

time: 53.7 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


time: 52.8 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

time: 99.5 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

time: 53.9 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f833af6bf50>,
 'gender': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f833af6c750>,
 'age': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f833af6a5d0>,
 'occupation': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f83adda6690>,
 'zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f833af63850>,
 'hist_movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f833af734d0>,
 'hist_genres': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f833b4f6d90>,
 'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f833af73410>}

time: 58.1 ms (started: 2022-10-12 15:42:26 +00:00)


# Create User Inputs


## Create Inputs Tensor

In [ ]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

time: 99.6 ms (started: 2022-10-12 15:42:26 +00:00)


In [ ]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'gender:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'age:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'occupation:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'zip:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_movie_id:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_genres:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len:0' shape=(None, 1) dtype=int32>]

time: 52.4 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


time: 61.9 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
item_inputs_list

[<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>]

time: 55 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
item_inputs_list[0]

<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>

time: 80.4 ms (started: 2022-10-12 16:27:16 +00:00)


## Embed Inputs

In [ ]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

time: 115 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

time: 90.7 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

time: 52 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

time: 51.5 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_gender/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_age/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_occupation/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_zip/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_1/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

time: 55.4 ms (started: 2022-10-12 15:42:27 +00:00)


## Combine Embeded Input (User only)

In [ ]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

time: 44.4 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 144) dtype=float32>

time: 73.3 ms (started: 2022-10-12 15:42:27 +00:00)


# Sepecify Model Layers

### User Layers

In [ ]:
from tensorflow.keras import layers

time: 46 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

time: 47.5 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
from deepctr.layers import DNN

time: 63.3 ms (started: 2022-10-12 15:42:27 +00:00)


In [ ]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



time: 52.7 ms (started: 2022-10-12 15:42:28 +00:00)


In [ ]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 756 ms (started: 2022-10-12 15:42:28 +00:00)


### Item Layer

In [ ]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(3019, 32) dtype=float32>

time: 308 ms (started: 2022-10-12 15:42:28 +00:00)


In [ ]:
data[item_feature_name].min(), data[item_feature_name].max()

(1, 3018)

time: 97.8 ms (started: 2022-10-12 17:08:19 +00:00)


In [ ]:
item_vocabulary_size, item_index.eval(session=K.get_session()).shape

(3019, (3019,))

time: 48.4 ms (started: 2022-10-12 17:03:53 +00:00)


In [258]:
item_features[item_feature_name]

<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>

time: 117 ms (started: 2022-10-12 18:07:25 +00:00)


In [ ]:
item_embedding_weight.eval(session=K.get_session()).shape

(3019, 32)

time: 178 ms (started: 2022-10-12 17:02:02 +00:00)


In [ ]:
item_embs.shape

(3018, 32)

time: 164 ms (started: 2022-10-12 17:01:58 +00:00)


In [ ]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(3019, 32)

time: 323 ms (started: 2022-10-12 15:42:29 +00:00)


In [ ]:
# .eval(session=K.get_session())

time: 50.1 ms (started: 2022-10-12 15:42:29 +00:00)


### Output Layer

In [ ]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

time: 240 ms (started: 2022-10-12 15:42:29 +00:00)


In [ ]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

time: 47.6 ms (started: 2022-10-12 15:42:29 +00:00)


In [ ]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

time: 47.4 ms (started: 2022-10-12 15:42:29 +00:00)


In [ ]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(3019, 32) dtype=float32>

time: 54.8 ms (started: 2022-10-12 15:42:30 +00:00)


In [ ]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 56.7 ms (started: 2022-10-12 15:42:30 +00:00)


In [ ]:
item_features[item_feature_name]

<tf.Tensor 'movie_id:0' shape=(None, 1) dtype=int32>

time: 59.4 ms (started: 2022-10-12 15:42:30 +00:00)


## Build Model

In [ ]:
from tensorflow.python.keras.models import Model

time: 48.5 ms (started: 2022-10-12 15:42:30 +00:00)


In [ ]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


time: 67.1 ms (started: 2022-10-12 15:42:30 +00:00)


# Model Compile and Training

In [ ]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

time: 207 ms (started: 2022-10-12 15:42:30 +00:00)


In [ ]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 11802 samples
Epoch 1/20
11802/11802 [==============================] - 2s 135us/sample - loss: 5.1564
Epoch 2/20
11802/11802 [==============================] - 1s 82us/sample - loss: 5.0330
Epoch 3/20
11802/11802 [==============================] - 1s 77us/sample - loss: 5.0067
Epoch 4/20
11802/11802 [==============================] - 1s 73us/sample - loss: 5.0103
Epoch 5/20
11802/11802 [==============================] - 1s 79us/sample - loss: 5.0031
Epoch 6/20
11802/11802 [==============================] - 1s 76us/sample - loss: 4.9821
Epoch 7/20
11802/11802 [==============================] - 1s 78us/sample - loss: 4.9510
Epoch 8/20
11802/11802 [==============================] - 1s 76us/sample - loss: 4.9900
Epoch 9/20
11802/11802 [==============================] - 1s 79us/sample - loss: 4.9435
Epoch 10/20
11802/11802 [==============================] - 1s 77us/sample - loss: 4.9346
Epoch 11/20
11802/11802 [==============================] - 1s 77us/sample - loss: 4.9032
Epoch 

In [ ]:
res = model.predict(train_model_input)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


time: 2.27 s (started: 2022-10-12 15:42:50 +00:00)


# Predict: Generate Item and User Embeddings

In [ ]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(3, 32)
(3018, 32)
time: 196 ms (started: 2022-10-12 15:42:52 +00:00)


In [ ]:
item_profile['movie_id'].values.min(), item_profile['movie_id'].values.max()

(1, 3018)

time: 184 ms (started: 2022-10-12 17:16:25 +00:00)


In [ ]:
item_embs.shape

(3018, 32)

time: 79.1 ms (started: 2022-10-12 17:16:11 +00:00)


In [ ]:
user_embs.shape, item_embs.shape

((3, 32), (3018, 32))

time: 48.2 ms (started: 2022-10-12 15:42:52 +00:00)


In [ ]:
prob_matrix = user_embs.dot(item_embs.T)

time: 59.1 ms (started: 2022-10-12 15:42:52 +00:00)


In [ ]:
prob_matrix.shape

(3, 3018)

time: 79.2 ms (started: 2022-10-12 15:42:52 +00:00)


# Recommendation

In [ ]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 4.9 MB/s 
time: 4.73 s (started: 2022-10-12 15:42:52 +00:00)


In [ ]:
import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

time: 84.2 ms (started: 2022-10-12 15:42:57 +00:00)


In [ ]:
test_true_label = {line[0]:[line[1]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0
pred_label = {}
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['movie_id'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        pred_label[uid] = pred
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
print("recall", np.mean(s))
print("hit rate", hit / len(test_user_model_input['user_id']))

3it [00:00, 1368.90it/s]


recall 0.0
hit rate 0.0
time: 63.3 ms (started: 2022-10-12 15:42:58 +00:00)


In [ ]:
test_user_model_input['user_id']

array([2, 3, 1])

time: 52.8 ms (started: 2022-10-12 17:17:35 +00:00)


# Validation

In [ ]:
df_train = pd.DataFrame(train_set)

time: 73 ms (started: 2022-10-12 15:42:58 +00:00)


In [ ]:
df_train.head()

,0,1,2,3,4,5,6,7
0,3,290,0,"[2758, 1926, 2096, 1594, 1081, 2783, 1725, 756...",50,"[5, 5, 5, 6, 5, 8, 8, 6, 8, 8, 18, 5, 5, 8, 8,...",5,NaN
1,2,1511,1,"[1727, 2410, 1446, 2615, 2120, 2667, 2680, 820...",50,"[8, 8, 12, 6, 6, 2, 5, 16, 5, 5, 5, 3, 8, 5, 3...",4,3.0
2,3,843,1,"[1451, 225, 555, 1460, 1516, 960, 2248, 1555, ...",50,"[8, 8, 8, 5, 8, 5, 8, 8, 8, 1, 16, 5, 8, 5, 8,...",8,4.0
3,2,1335,1,"[822, 2965, 821, 244, 782, 1494, 1062, 2785, 2...",50,"[5, 8, 5, 6, 4, 1, 1, 1, 8, 8, 3, 8, 5, 8, 8, ...",1,4.0
4,2,756,1,"[1546, 890, 760, 2722, 1012, 423, 2496, 527, 1...",50,"[1, 5, 8, 16, 8, 1, 5, 8, 8, 8, 8, 8, 5, 5, 13...",6,2.0


time: 242 ms (started: 2022-10-12 17:18:41 +00:00)


In [ ]:
D

array([[0.8742792 , 0.84926516, 0.83438325, 0.7869048 , 0.78441876,
        0.75806993, 0.7518325 , 0.7442567 , 0.73508465, 0.7305696 ,
        0.72341657, 0.7137732 , 0.7122004 , 0.71010906, 0.70333654,
        0.6964692 , 0.6933807 , 0.69336516, 0.69186705, 0.69029033,
        0.68633604, 0.685551  , 0.682623  , 0.6782071 , 0.67726475,
        0.6760137 , 0.6737901 , 0.6706406 , 0.6695087 , 0.66835207,
        0.66578996, 0.6645329 , 0.66399306, 0.6634692 , 0.66307735,
        0.661651  , 0.66142607, 0.6611558 , 0.660971  , 0.6603552 ,
        0.660263  , 0.66022414, 0.65679586, 0.65597165, 0.6556176 ,
        0.6531124 , 0.6525485 , 0.64979464, 0.6484739 , 0.64292973],
       [0.83818305, 0.83219385, 0.80338234, 0.7999476 , 0.79457784,
        0.7686687 , 0.7628434 , 0.761816  , 0.7613894 , 0.75610334,
        0.7544562 , 0.75122267, 0.7450205 , 0.7440881 , 0.74211746,
        0.74013436, 0.73810923, 0.73610055, 0.7303264 , 0.7297689 ,
        0.72918624, 0.7219011 , 0.71752274, 0.7

time: 55 ms (started: 2022-10-12 17:36:08 +00:00)


In [278]:
user_idx = 1
uid = 3
sorted_index = prob_matrix.argsort()[:,-50:]
-np.sort(-prob_matrix[user_idx][sorted_index[user_idx]])

array([0.83818305, 0.83219385, 0.80338234, 0.7999477 , 0.7945778 ,
       0.7686687 , 0.7628434 , 0.761816  , 0.76138926, 0.7561032 ,
       0.7544563 , 0.75122267, 0.7450205 , 0.7440881 , 0.74211746,
       0.74013436, 0.73810923, 0.73610055, 0.7303264 , 0.7297688 ,
       0.72918624, 0.7219011 , 0.7175228 , 0.7132602 , 0.71223915,
       0.7112775 , 0.71019137, 0.7077772 , 0.70745456, 0.70402104,
       0.7018446 , 0.7000875 , 0.6995321 , 0.69562876, 0.6937429 ,
       0.69229263, 0.6921176 , 0.6912354 , 0.685327  , 0.68007016,
       0.6774778 , 0.6774292 , 0.67653126, 0.6753539 , 0.6743387 ,
       0.67122644, 0.66962856, 0.6658136 , 0.66403174, 0.6630513 ],
      dtype=float32)

time: 81.5 ms (started: 2022-10-12 18:15:50 +00:00)


In [279]:
np.array([item_profile['movie_id'].values[x] for x in I[user_idx]])
# I[0]

array([ 817, 2815, 2590, 2510,  523, 1314,  989, 1889, 2276, 2858, 2610,
       2103, 2612, 1823, 1441, 1515, 1095, 2883, 2759,  438, 1356, 1419,
       1886, 2674, 2460,  456, 1229, 2269,  732,  972, 1179, 1544, 1523,
       2851, 1759, 2280, 1453, 1415,  729,  203, 2026, 1161, 1529, 1044,
       2752, 2318, 1370, 2293, 2604, 2682])

time: 53.5 ms (started: 2022-10-12 18:15:56 +00:00)


In [280]:
I[user_idx]

array([2829,  545, 2497, 2941, 1414, 1689,  446,  312, 2236, 1724,  958,
       1071,  939, 1820, 2199,  553, 2702, 1111, 2883, 2840, 2878, 1176,
       1779,  292,  874,  548, 2752, 1340, 2774, 1981, 1687, 1776,  249,
       1139,  593,  935,  915,  956, 2624, 1821, 1562, 2620, 1346, 1159,
       1315, 1484, 2715, 1729, 2487, 2886])

time: 49.2 ms (started: 2022-10-12 18:15:57 +00:00)


In [281]:
(sorted_index[user_idx])[::-1]

array([2829,  545, 2497, 2941, 1414, 1689,  446,  312, 2236, 1724,  958,
       1071,  939, 1820, 2199,  553, 2702, 1111, 2883, 2840, 2878, 1176,
       1779,  292,  874,  548, 2752, 1340, 2774, 1981, 1687, 1776,  249,
       1139,  593,  935,  915,  956, 2624, 1821, 1562, 2620, 1346, 1159,
       1315, 1484, 2715, 1729, 2487, 2886])

time: 51 ms (started: 2022-10-12 18:15:57 +00:00)


In [282]:
len(set(df_train[df_train[0]==uid][1].unique()) & set(sorted_index[user_idx]))

47

time: 51.4 ms (started: 2022-10-12 18:16:00 +00:00)


In [283]:
len(set(df_train[df_train[0]==uid][1].unique()) & set([item_profile['movie_id'].values[x] for x in I[user_idx]]))

50

time: 59.7 ms (started: 2022-10-12 18:16:01 +00:00)


In [277]:
sorted_index[user_idx]

array([1409, 2620, 1708, 1511,  868,   21,  653,  547, 1014,  917,  817,
        785, 1846, 1146,  747, 1482,  990, 1408, 1755, 3013,  605,  506,
       1275, 1036,  662,  376,  193, 2815,  819,  992, 1472, 2814, 1696,
       1515,  776, 1812, 1694,  758, 1253, 1438,  385, 1128,  214, 2988,
       1269, 1441, 1471, 1530, 1287, 1240])

time: 58 ms (started: 2022-10-12 18:14:57 +00:00)
